In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    data = []
    
    pagination = soup.find("div", class_="styles_pagination__6VmQv")
    last_page = pagination.find_all("a")[-2].text
    last_page = pd.to_numeric(last_page)
    
    for i in range(1, last_page + 1):
        page_url = url + "?page=" + str(i)
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        reviews = page_soup.find_all("div", class_="styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ")
        
        for review in reviews:
            review_data = {}
            
            review_data["Company"] = soup.find("h1", class_="typography_default__hIMlQ typography_appearance-default__AAY17 title_title__i9V__").span.get_text(strip=True)
            review_data["Customer"] = review.find("span", class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
            review_data["Number_review"] = review.find("div", class_="styles_consumerExtraDetails__fxS4S").find("span").text
            review_data["Language"] = review.find("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua").text
            review_data["Title"] = review.find("h2", class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
            review_data["Date_review"] = review.find("div", class_="styles_reviewHeader__iU9Px").time.text
            reply = review.find("div").find("p", class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX")
            review_data['Reply'] = reply.text if reply else None
            review_data['Date_reply'] = review.find("div", class_="styles_content__Hl2Mi").time.text if reply else None
            review_data["Rating"] = review.find("section", class_="styles_reviewContentwrapper__zH_9M").div["data-service-review-rating"]
            type_element = review.find_next("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__yqwWi")
            review_data["Status"] = type_element.find("span").text if type_element else None
            review_data["Experience Date"] = review.find("div", class_="styles_reviewContent__0Q2Tg").text
            
            data.append(review_data)
    
    df = pd.DataFrame(data)
    return df


In [2]:
younited_url = "https://www.trustpilot.com/review/www.younited-credit.com"
df_younited = extract_reviews(younited_url)
#df_younited.to_csv("younited.csv")

In [3]:
cofidis_url = "https://www.trustpilot.com/review/www.cofidis.fr"
df_cofidis = extract_reviews(cofidis_url)
#df_cofidis.to_csv("cofidis.csv")

In [4]:
floa_url = "https://www.trustpilot.com/review/www.floabank.fr"
df_floa = extract_reviews(floa_url)
#df_floa.to_csv("floa.csv")


In [5]:
orange_url = "https://www.trustpilot.com/review/www.orangebank.fr"
df_orange = extract_reviews(orange_url)
#df_orange.to_csv("orange.csv")

In [6]:
bourso_url = "https://www.trustpilot.com/review/boursorama-banque.com"
df_bourso = extract_reviews(bourso_url)
#df_bourso.to_csv("bourso.csv")

In [7]:
anytime_url = "https://www.trustpilot.com/review/anyti.me"
df_anytime = extract_reviews(anytime_url)
#df_anytime.to_csv("anytime.csv")

In [8]:
df_all = pd.concat([df_younited, df_cofidis, df_orange, df_floa, df_bourso, df_anytime], axis=0)

df_all = df_all.reset_index(drop=True)
df_all

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience Date
0,Younited Credit,morris,1 review,FR,LA DEMARCHE INTERNET EST SIMPLE ET…,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,LA DEMARCHE INTERNET EST SIMPLE ET…LA DEMARCHE...
1,Younited Credit,Sall,3 reviews,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T..."
2,Younited Credit,cliente,2 reviews,FR,Tres bon contact,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,Tres bon contactA l écoute et bon contactDate ...
3,Younited Credit,Auré,1 review,FR,Rapide,"Jun 12, 2023","Bonjour,Merci d’avoir pris le temps de nous fa...","Jun 13, 2023",5,Verified,"RapideRapide, simple et efficaceDate of experi..."
4,Younited Credit,Lazarus Jamison,1 review,MW,I am yet to confirm the efficiency and…,"May 6, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","May 10, 2023",5,Verified,I am yet to confirm the efficiency and…I am ye...
...,...,...,...,...,...,...,...,...,...,...,...
2409,Anytime,RAPHAEL N,1 review,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...
2410,Anytime,FREDERIC LEGOUGE,7 reviews,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................
2411,Anytime,Mokhtar,2 reviews,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellentDate of experience: July...
2412,Anytime,JEAN-LOUIS CATALDO,2 reviews,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudiéDat...


In [9]:
df_all

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience Date
0,Younited Credit,morris,1 review,FR,LA DEMARCHE INTERNET EST SIMPLE ET…,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,LA DEMARCHE INTERNET EST SIMPLE ET…LA DEMARCHE...
1,Younited Credit,Sall,3 reviews,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T..."
2,Younited Credit,cliente,2 reviews,FR,Tres bon contact,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,Tres bon contactA l écoute et bon contactDate ...
3,Younited Credit,Auré,1 review,FR,Rapide,"Jun 12, 2023","Bonjour,Merci d’avoir pris le temps de nous fa...","Jun 13, 2023",5,Verified,"RapideRapide, simple et efficaceDate of experi..."
4,Younited Credit,Lazarus Jamison,1 review,MW,I am yet to confirm the efficiency and…,"May 6, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","May 10, 2023",5,Verified,I am yet to confirm the efficiency and…I am ye...
...,...,...,...,...,...,...,...,...,...,...,...
2409,Anytime,RAPHAEL N,1 review,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...
2410,Anytime,FREDERIC LEGOUGE,7 reviews,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................
2411,Anytime,Mokhtar,2 reviews,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellentDate of experience: July...
2412,Anytime,JEAN-LOUIS CATALDO,2 reviews,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudiéDat...


In [10]:
df_all.isnull().sum()

Company              0
Customer             0
Number_review        0
Language             0
Title                0
Date_review          0
Reply              960
Date_reply         960
Rating               0
Status              98
Experience Date      0
dtype: int64

In [11]:
df_all[["Number_review", "review"]]= df_all["Number_review"].str.split(" ", n = 1, expand = True)
df_all[["Number_review", "review"]]

,Number_review,review
0,1,review
1,3,reviews
2,2,reviews
3,1,review
4,1,review
...,...,...
2409,1,review
2410,7,reviews
2411,2,reviews
2412,2,reviews


In [12]:
df_all[["ExperienceDate", "Date_experience"]]= df_all["Experience Date"].str.split(":", n = 1, expand = True)

In [13]:
df_all[["Experience", "Date"]]= df_all["Experience Date"].str.split("Date", n = 1, expand = True)
df_all['Experience'][2]

'Tres bon contactA l écoute et bon contact'

In [14]:
new_df = df_all[["Company", "Customer", "Number_review", "Language", "Title", "Date_review", "Reply", "Date_reply", "Rating", "Status", "Experience", "Date_experience"]]
new_df

,Company,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
0,Younited Credit,morris,1,FR,LA DEMARCHE INTERNET EST SIMPLE ET…,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,LA DEMARCHE INTERNET EST SIMPLE ET…LA DEMARCHE...,"June 05, 2023"
1,Younited Credit,Sall,3,FR,Procedure top,"May 30, 2023","Bonjour, Nous vous remercions pour votre comme...","May 31, 2023",4,Verified,"Procedure topProcedure top, simple et rapide.T...","May 21, 2023"
2,Younited Credit,cliente,2,FR,Tres bon contact,3 days ago,"Bonjour,Merci d’avoir pris le temps de nous fa...",2 days ago,5,Verified,Tres bon contactA l écoute et bon contact,"June 08, 2023"
3,Younited Credit,Auré,1,FR,Rapide,"Jun 12, 2023","Bonjour,Merci d’avoir pris le temps de nous fa...","Jun 13, 2023",5,Verified,"RapideRapide, simple et efficace","June 06, 2023"
4,Younited Credit,Lazarus Jamison,1,MW,I am yet to confirm the efficiency and…,"May 6, 2023","Bonjour,Nous vous remercions d’avoir pris le t...","May 10, 2023",5,Verified,I am yet to confirm the efficiency and…I am ye...,"April 06, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...
2409,Anytime,RAPHAEL N,1,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"Sep 5, 2015",None,None,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,"September 05, 2015"
2410,Anytime,FREDERIC LEGOUGE,7,FR,CARTE ANYTIME,"Jul 14, 2015",None,None,5,Invited,CARTE ANYTIMESuper ..............................,"July 14, 2015"
2411,Anytime,Mokhtar,2,FR,Anytime,"Jul 13, 2015",None,None,5,Invited,AnytimeSuper excellent,"July 13, 2015"
2412,Anytime,JEAN-LOUIS CATALDO,2,FR,Très intéressant,"Jul 13, 2015",None,None,4,Invited,Très intéressantUn système très bien étudié,"July 13, 2015"


In [15]:
new_df.groupby(by="Status").describe()

Company                              Customer                       \
             count unique              top freq    count unique      top freq   
Status                                                                          
Invited       1772      6      Orange Bank  721     1772   1654   Client   10   
Redirected      12      1         Floabank   12       12     12  Vicky F    1   
Verified       532      2  Younited Credit  530      532    410  cliente   55   

           Number_review         ... Rating       Experience         \
                   count unique  ...    top  freq      count unique   
Status                           ...                                  
Invited             1772     24  ...      5  1459       1772   1533   
Redirected            12      4  ...      5     9         12     12   
Verified             532     12  ...      5   457        532    446   

                                                                    \
                                                          top freq   
Status                                                               
Invited                                      Simple et rapide   29   
Redirected  Clearly the staff don't work on commission!Jus...    1   
Verified                                     Simple et rapide   22   

           Date_experience                                  
                     count unique                 top freq  
Status                                                      
Invited               1772    710   February 22, 2022   29  
Redirected              12     12   February 28, 2023    1  
Verified               532    430     August 09, 2020    7  

[3 rows x 44 columns]

In [16]:
new_df[["Reply", "Date_reply"]].isna().sum()


Reply         960
Date_reply    960
dtype: int64

In [17]:
new_df[["Status"]] = new_df[["Status"]].fillna(method = "pad")
new_df[["Status"]]

C:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Status
0,Verified
1,Verified
2,Verified
3,Verified
4,Verified
...,...
2409,Invited
2410,Invited
2411,Invited
2412,Invited


In [18]:

#new_df[new_df["Reply"] == "No Reply"]

In [19]:
new_df.isna().sum()

Company              0
Customer             0
Number_review        0
Language             0
Title                0
Date_review          0
Reply              960
Date_reply         960
Rating               0
Status               0
Experience           0
Date_experience      0
dtype: int64

In [20]:
new_df.dtypes

Company            object
Customer           object
Number_review      object
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating             object
Status             object
Experience         object
Date_experience    object
dtype: object

In [21]:
new_df[["Number_review", "Rating"]] = new_df[["Number_review", "Rating"]].astype("int64")
new_df.dtypes

C:\Users\yaya-\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Company            object
Customer           object
Number_review       int64
Language           object
Title              object
Date_review        object
Reply              object
Date_reply         object
Rating              int64
Status             object
Experience         object
Date_experience    object
dtype: object

In [22]:
new_df[["Date_review", "Date_reply", "Date_experience"]] = new_df[["Date_review", "Date_reply","Date_experience"]].apply(pd.to_datetime, errors='coerce')
# Printing the converted datetime object
new_df.isnull().sum()


Company               0
Customer              0
Number_review         0
Language              0
Title                 0
Date_review          42
Reply               960
Date_reply         1006
Rating                0
Status                0
Experience            0
Date_experience      29
dtype: int64

In [23]:
new_df.dtypes

Company                    object
Customer                   object
Number_review               int64
Language                   object
Title                      object
Date_review        datetime64[ns]
Reply                      object
Date_reply         datetime64[ns]
Rating                      int64
Status                     object
Experience                 object
Date_experience    datetime64[ns]
dtype: object

In [24]:
new_df = new_df.set_index("Company")

In [25]:
new_df[["Reply"]] = new_df[["Reply"]].fillna("No Reply")
new_df[["Reply"]]

,Reply
Company,
Younited Credit,"Bonjour,Merci d’avoir pris le temps de nous fa..."
Younited Credit,"Bonjour, Nous vous remercions pour votre comme..."
Younited Credit,"Bonjour,Merci d’avoir pris le temps de nous fa..."
Younited Credit,"Bonjour,Merci d’avoir pris le temps de nous fa..."
Younited Credit,"Bonjour,Nous vous remercions d’avoir pris le t..."
...,...
Anytime,No Reply
Anytime,No Reply
Anytime,No Reply


In [26]:
satisfaction_client = new_df
satisfaction_client

,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
Company,,,,,,,,,,,
Younited Credit,morris,1,FR,LA DEMARCHE INTERNET EST SIMPLE ET…,NaT,"Bonjour,Merci d’avoir pris le temps de nous fa...",NaT,5,Verified,LA DEMARCHE INTERNET EST SIMPLE ET…LA DEMARCHE...,2023-06-05
Younited Credit,Sall,3,FR,Procedure top,2023-05-30,"Bonjour, Nous vous remercions pour votre comme...",2023-05-31,4,Verified,"Procedure topProcedure top, simple et rapide.T...",2023-05-21
Younited Credit,cliente,2,FR,Tres bon contact,NaT,"Bonjour,Merci d’avoir pris le temps de nous fa...",NaT,5,Verified,Tres bon contactA l écoute et bon contact,2023-06-08
Younited Credit,Auré,1,FR,Rapide,2023-06-12,"Bonjour,Merci d’avoir pris le temps de nous fa...",2023-06-13,5,Verified,"RapideRapide, simple et efficace",2023-06-06
Younited Credit,Lazarus Jamison,1,MW,I am yet to confirm the efficiency and…,2023-05-06,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-05-10,5,Verified,I am yet to confirm the efficiency and…I am ye...,2023-04-06
...,...,...,...,...,...,...,...,...,...,...,...
Anytime,RAPHAEL N,1,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05,No Reply,NaT,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05
Anytime,FREDERIC LEGOUGE,7,FR,CARTE ANYTIME,2015-07-14,No Reply,NaT,5,Invited,CARTE ANYTIMESuper ..............................,2015-07-14
Anytime,Mokhtar,2,FR,Anytime,2015-07-13,No Reply,NaT,5,Invited,AnytimeSuper excellent,2015-07-13


In [27]:
satisfaction_client.isnull().sum()

Customer              0
Number_review         0
Language              0
Title                 0
Date_review          42
Reply                 0
Date_reply         1006
Rating                0
Status                0
Experience            0
Date_experience      29
dtype: int64

In [28]:
satisfaction_client

,Customer,Number_review,Language,Title,Date_review,Reply,Date_reply,Rating,Status,Experience,Date_experience
Company,,,,,,,,,,,
Younited Credit,morris,1,FR,LA DEMARCHE INTERNET EST SIMPLE ET…,NaT,"Bonjour,Merci d’avoir pris le temps de nous fa...",NaT,5,Verified,LA DEMARCHE INTERNET EST SIMPLE ET…LA DEMARCHE...,2023-06-05
Younited Credit,Sall,3,FR,Procedure top,2023-05-30,"Bonjour, Nous vous remercions pour votre comme...",2023-05-31,4,Verified,"Procedure topProcedure top, simple et rapide.T...",2023-05-21
Younited Credit,cliente,2,FR,Tres bon contact,NaT,"Bonjour,Merci d’avoir pris le temps de nous fa...",NaT,5,Verified,Tres bon contactA l écoute et bon contact,2023-06-08
Younited Credit,Auré,1,FR,Rapide,2023-06-12,"Bonjour,Merci d’avoir pris le temps de nous fa...",2023-06-13,5,Verified,"RapideRapide, simple et efficace",2023-06-06
Younited Credit,Lazarus Jamison,1,MW,I am yet to confirm the efficiency and…,2023-05-06,"Bonjour,Nous vous remercions d’avoir pris le t...",2023-05-10,5,Verified,I am yet to confirm the efficiency and…I am ye...,2023-04-06
...,...,...,...,...,...,...,...,...,...,...,...
Anytime,RAPHAEL N,1,FR,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05,No Reply,NaT,5,Invited,ANYTIME IS NOT A BANK BUT THEY HAVE DONE A GRE...,2015-09-05
Anytime,FREDERIC LEGOUGE,7,FR,CARTE ANYTIME,2015-07-14,No Reply,NaT,5,Invited,CARTE ANYTIMESuper ..............................,2015-07-14
Anytime,Mokhtar,2,FR,Anytime,2015-07-13,No Reply,NaT,5,Invited,AnytimeSuper excellent,2015-07-13


In [29]:
satisfaction_client.shape

(2414, 11)